<h1><center> Non-parametric comparison of convolutional neural networks and CaiT transformer in the classification of COVID-19 in chest CT scans </center></h1>


<h2>Table of Contents</h2>
    <ul>
    <li><a href="#Section_1">Import libraries and supporting functions </a></li>
    <li><a href="#Section_2"> Build the main comparison function</a> </li>
    <li><a href="#Section_3">Define parameters and run the comparison</a></li>
    <li><a href="#Section_4"> Extract the result in LaTeX tables </a></li>
    </ul>

<h2 id="#Section_1">Import libraries and supporting functions </h2>

__Here we first import the libraries and functions required to run the comparison__

In [ ]:
import os
import pandas as pd
from stats.non_parametric_stats import *
from stats.utils import df_to_latex
import warnings
%matplotlib inline
warnings.simplefilter(action='ignore', category=FutureWarning)

sns.set(style='whitegrid')


<h2 id="Section_2"> Build the main comparison function </h2>

Here we extract information from the row data and start the comparison process

In [ ]:
def main(**params):
    """
    :param params: a dictionary to access the data runs, group the data by
    experiment or by network architecture, num of bootstrap samples, plot and
    save plots and dataframes in csv format. Besides, it provides a summary
    of the accuracies at training, validation and test. It also summarizes of
    all performance metrics, before and after bootstrapping. The posthoc
    friedman-nemenyi test stats also are in the dictionary for all performance
    metrics at test(accuracy, balanced accuracy, F1, F2, MCC, sensibility, specificity).
    :return: a dictionary with the summary of the accuracy during train, validation and test
    results, evaluation metrics and probabilities.
    """
    # Raise an assertion if incorrect grouping
    assert params['group'] in ['Architecture', 'Experiment'], 'Group by Architecture or Experiment'

    # Save output in a dictionary
    outputs = {}

    # Read data
    df = pd.read_csv(os.path.join(params['root'], params['data']))

    # Sorting by run architecture, loss and optimizer
    df = df.sort_values(['Run', 'Architecture', 'Loss', 'Optimizer'])

    # Abbreviate architecture name for plotting purpose
    df = df.replace({'MobileNet-v3-large': 'MobileNet-v3'})

    # Rename columns
    df = df.rename(columns={'Exp': 'Experiment', 'Sensitivity': 'Sens',
                            'F1 macro': 'F1', 'Specificity': 'Spec', 'Max epoch': 'Epoch'})
    # Re-number experiment as Exp-xx
    df['Experiment'] = ['Exp-' + str(x).zfill(2) for x in df['Experiment']]

    #  Create the experimental set-up
    df_set = df[['Experiment', 'Architecture', 'Loss', 'Optimizer']][0:20].set_index('Experiment')
    df_set = df_set.sort_values(by='Experiment')
    outputs['exp_setup'] = df_set

    # Filter data by training, validation and test accuracies
    df_acc = df.rename(columns={'Accuracy': 'Test acc', 'val_acc': 'Val acc',
                                'tr_acc': 'Train acc'})

    # Create directories to save figures and csv files
    outdir = os.path.join(params['root'], 'figures', params['group'].lower())
    csv_dir = os.path.join(params['root'], 'csv_files', params['group'].lower())

    # Create directories if they don't exist
    dirs = [outdir, csv_dir]
    for d in dirs:
        if not os.path.exists(d):
            os.makedirs(d)
            print('Directory created')

    # Dataframe summarising train,test and validation accuracies
    acc_sum = acc_summary(params['group'], params['acc'], df_acc, outdir, csv_dir, 
                          params['boxplot'], params['hideplot'])
    # Save results in a dictionary
    outputs['accuracies'] = acc_sum

    # Filtering to get data only for the metrics to be compared
    df_covid = df.filter(['Experiment', 'Architecture', 'Loss', 'Optimizer', 'Accuracy', 'BA',
                          'MCC', 'F1', 'F2', 'Sens', 'Spec', 'Max acc', 'Epoch'])
    df_covid = df_covid.rename(columns={'Accuracy': 'Acc'})

    # Provide a summary of all evaluation metrics before bootstrapping
    if params['stats_sum']:
        st_s = stats_summary(params['group'], params['metrics'], df_covid, csv_dir)
        outputs['pre_boots'] = st_s

    # Bootstrapping summary and post-hoc test
    ranks, intervals, post_hoc = bootstrap_stats_summary(params['group'], params['metrics'], 
                                                         df_covid, params['n_bootstraps'], 
                                                         outdir, csv_dir, params['alpha'], 
                                                         params['hideplot'],  params['nemenyi'])
    # Save into the dictionary
    outputs['ranks'], outputs['ci'] = ranks, intervals
    outputs['stats_comp'] = post_hoc

    # Bootstrap for maximum acc and epoch - training
    epochs, max_acc = bootstrapping_epochs(params['n_bootstraps'], df_covid,
                                           'Epoch', 'Max acc', params['group'])

    # boxplot for bootstrapped epoch with max accuracy and accuracy during validation
    mx_rank, ci_pval, epochs_acc = boots_epochs_df(epochs, max_acc, params['group'], outdir, 
                                                   csv_dir, params['boxplot'], 
                                                   params['hideplot'], nemenyi=params['nemenyi'])

    # Save into the dictionary max accuracy ranking, pvalues and confidence intervals
    outputs['max_rank'], outputs['max_acc_stats'] = mx_rank, ci_pval

    # density distribution for bootstrapped max accuracy and number of epochs during validation
    if params['dist_plot']:
        for a in ['epochs', 'max acc']:
            max_acc_epoch_plot(epochs_acc, params['group'], col_name=a, outdir=outdir,
                               hideplot=params['hideplot'])

    # Extract the friedman statistic and associated p-value
    friedman_test = {k: v[0:2] for (k, v) in post_hoc.items()}

    # Update dictionary to include Friedman-Nemenyi stats for the maximum accuracy and the number
    # of training epochs
    friedman_test.update({k: v for k, v in ci_pval.items() if k == 'Max accuracy' or
                          k == 'Epochs'})
    pval = pd.DataFrame.from_dict(friedman_test).T.reset_index()
    pval.columns = ['Metric', 'Friedman', 'p-value']

    # Put together all post-hoc stats and p-value into the dictionary
    outputs['pval'] = pval

    # save dataframe
    pval.to_csv(os.path.join(csv_dir, params['group'].lower() + '_pval.csv'))

    return outputs


<h2 id="Section_3">Define parameters and run the comparison </h2>

In [ ]:
if __name__ == '__main__':
    
    acc=['Train acc','Val acc','Test acc']
    metrics = ['Acc','BA','F1', 'F2','MCC','Sens','Spec']
    
    args = {'root': '/Users/aze_ace/Documents/pythonProject/covid_project', 'data':'covid_10_runs'
                                                                                   '.csv',
        'acc':acc, 'stats_sum': True, 'group': 'Architecture', 'metrics': metrics,'n_bootstraps': 1000,
        'alpha': 5.0, 'nemenyi': True, 'hideplot': True, 'boxplot': True, 'dist_plot': True}
    
    sum_net= main(args)
    
    args['group'] = 'Experiment'
    
    sum_exp = main(args)
    
    prob_bar(os.path.join(args['root'], 'figures'))
    
 

<h2 id="Section_3"> Extract the result in LaTeX tables  </h2>

In [ ]:
# Define path to save the tables and create 

path = '/Users/aze_ace/Documents/pythonProject/covid_project/tables'
if not os.path.exists(path):
    os.makedirs(path)

### a) Experimental setup table

In [ ]:
f_setup='ex_setup'
caption= 'Experimetal design'
label= 'tab: exp_setup'
df1= sum_exp['exp_setup']
df_to_latex(f_setup, caption, label, df1, path)

### b) Results tables by experiment

In [ ]:
f_acc='ex_acc1'
caption= 'Train, validation and test accuracies by experiment.'
label= 'tab: exp_acc'
df2= sum_exp['accuracies']
df_to_latex(f_acc, caption, label, df2, path)

In [ ]:
f_nacc='ex_metrics'
caption= 'Evaluation metrics by experiment before bootstrapping.'
label= 'tab: exp_metrics'
df3= sum_exp['pre_boots']
df_to_latex(f_nacc , caption, label, df3, path)

In [ ]:
f_rank='ex_rank'
caption= 'Ranks and median by experiment after bootstrapping.'
label= 'tab: exp_metrics'
df4= sum_exp['ranks']
df_to_latex(f_rank , caption, label, df4, path)

In [ ]:
f_ci = 'ex_ci'
caption= 'Confidence interval by experiment after bootstrapping.'
label= 'tab: expci'
df5= sum_exp['ci']
df_to_latex(f_ci, caption, label, df5, path)

In [ ]:
f_maxacc='ex_epoch'
caption= 'Ranking of the maximum training accuracy and training epochs with medians by experiment'
label= 'tab: exp_epoch'
df6= sum_exp['max_rank']
df_to_latex(f_maxacc, caption, label, df6, path)

### c)  Results tables by architechture

In [ ]:
f_acc='net_acc'
caption= 'Train, validation and test accuracies by architecture.'
label= 'tab: net_acc'
df7= sum_net['accuracies']
df_to_latex(f_acc, caption, label, df7)

In [ ]:
f_nacc='net_metrics'
caption= 'Evaluation metrics by architecture before bootstrapping.'
label= 'tab: net_metrics'
df8= sum_net['pre_boots']
df_to_latex(f_nacc , caption, label, df8)

In [ ]:
f_rank='net_rank'
caption= 'Ranks and median by architecture after bootstrapping.'
label= 'tab: net_metrics'
df9= sum_net['ranks']
df_to_latex(f_rank , caption, label, df9, path)

In [ ]:
f_ci = 'net_ci'
caption= 'Confidence interval by architecture after bootstrapping.'
label= 'tab: net_ci'
df10= sum_net['ci']
df_to_latex(f_ci, caption, label, df10, path)

In [ ]:
f_maxacc='net_epoch'
caption= 'Ranking of the maximum training accuracy and training epochs with medians by architecture.'
label= 'tab: net_epoch'
df11= sum_net['max_rank']
df_to_latex(f_maxacc, caption, label, df11, path)

### d) Friedman statistic and p-value table

In [ ]:
pd.options.display.float_format = "{:.3f}".format
df_pval=sum_net['pval']
df_pval_exp=sum_exp['pval']

In [ ]:
pval_net_exp= pd.concat([df_pval, df_pval_exp.iloc[:, 1:]], axis =1)

In [ ]:
headers =  ['Metric','Friedman-Arch', 'pval-Arch', 'Friedman-Exp', 'pval-Exp']
pval_net_exp.columns = headers

In [ ]:
df12 = pval_net_exp.set_index('Metric')

In [ ]:
f_nacc='Friedman'
caption= 'Friedman statistic and p-values by architectures and experiments at a confidence level alpha =0.05.'
label= 'tab: p-val'
df_to_latex(f_nacc , caption, label, df12, path)